# Analysing water levels of the river Isar

In light of the recent flooding catastrophe with more then 170 dead, my interest sparked in monitoring and maybe predicting the waterlevels of the river in my hometown Munich, the river Isar.

In this project you will follow how i refine my workflow and the identification of interesting data data. I will also improve my skills in webscraping with beautiful soup and time series analysis.

The project is structured in three main parts:

1. Problem formulation and subject understanding
2. Datasource identification and webscraping
3. Data wrangling and time series analysis

## 0. Loading packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from datetime import datetime, timedelta
from datetime import date

import os
dirname = os.getcwd()


## 1. Problem formulation and subject understanding

The isar is a 292,3 kilometer long river that originates in the very north of Austria. The river crosses the capital of bavaria Munich until it merges with the Donau river south of Deggendorf. [Source: wikipedia.com/isar]



## 2. Data Source identification and webscraping

The bavarian ministry for envoironment runs the so called "Hochwassernachrichtendienst" HND where the water levels of several rivers and lakes are provided. Additionally there are information on precipication and others.

We focus on scraping the water levels first. The site is structured as follows

    Basic Link:           https://www.hnd.bayern.de/pegel/meldestufen/isar/tabellen
    Additional Parameter: ?days=0&hours=1

We can address the levels per day for the last 30 days while 0 is today and 29 the oldest. The data is provided on an hourly interval from 0 to 23. For now, we don't need the most recent data, but this might change when upgrading to a more sophisticated scraping approach. At the moment we focus on getting a basic scrapping to work.

In [2]:
#we create two lists for the days and hours we want to scrape.
days_scrape  = [i for i in range(1,30)]
hours_scrape = [i for i in range(0,24)]

In [3]:
link = "https://www.hnd.bayern.de/pegel/meldestufen/isar/tabellen?days=0&hours=1"
basic_link = "https://www.hnd.bayern.de/pegel/meldestufen/isar/tabellen"
dfs = pd.read_html(link)

As you can see, pandas 'read_html' class is a very powerful tool to get html tabels quickly into a dataframe. We now automate this approach to scrape the full history of the water levels.

In [4]:
df_water_levels_scrape = pd.DataFrame()

for day in days_scrape:
    for hour in hours_scrape:
        scrape_link = basic_link + '?days=' + str(day) + '&hours=' + str(hour)
        water_level = pd.read_html(scrape_link)
        df_water_levels_scrape = df_water_levels_scrape.append(water_level[0])

In [5]:
df_water_levels_scrape['Datum Zeit'] = pd.to_datetime(df_water_levels_scrape['Datum Zeit'])

In [6]:
df_water_levels_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22272 entries, 0 to 31
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Messstelle                 22272 non-null  object        
 1   Gewässer                   22272 non-null  object        
 2   Datum Zeit                 22272 non-null  datetime64[ns]
 3   Wasser­stand [cm]          22272 non-null  int64         
 4   Änderung seit 2 Std. [cm]  22272 non-null  int64         
 5   Abfluss [m³/s]             22272 non-null  object        
 6   Melde­stufe                22272 non-null  int64         
 7   Jähr­lichkeit              22272 non-null  object        
 8   Vorher­sage                0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.7+ MB


In [7]:
von_string = str(date.today() - timedelta(days_scrape[-1]))
bis_string = str(date.today())
path = dirname + "/ScrapingData/"

save_string = 'isar_pegel' + '_' + von_string +  '_bis_' + bis_string

df_water_levels_scrape.to_csv(path + save_string + '.csv')

In [8]:
df_water_levels_scrape.tail()

,Messstelle,Gewässer,Datum Zeit,Wasser­stand [cm],Änderung seit 2 Std. [cm],Abfluss [m³/s],Melde­stufe,Jähr­lichkeit,Vorher­sage
27,Berg,Sempt,2021-10-07 23:00:00,68,0,361,0,---,NaN
28,Eschenlohe Brücke,Loisach,2021-10-07 23:00:00,153,7,225,0,---,NaN
29,Starnberg,Starnberger See,2021-10-07 23:00:00,58446,1,---,0,---,NaN
30,Eching,Windach,2021-10-07 23:00:00,134,-1,730,0,---,NaN
31,Oberfinning Speicherabgabe,Windach,2021-10-07 23:00:00,74,0,373,0,---,NaN
